In [1]:
# Install needed libraries
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!pip install datasets

In [ ]:
pip install --upgrade trl

# nếu lỗi ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers'

In [4]:
from google.colab import drive
from datasets import load_dataset

# Gắn Google Drive vào Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
                          HfArgumentParser, TrainingArguments, pipeline, logging)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

#https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [6]:
# Checking and formating data set
# data full: https://huggingface.co/datasets/timdettmers/openassistant-guanaco
# format text: https://huggingface.co/datasets/mlabonne/guanaco-llama2?row=0
# data train: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k

from datasets import load_dataset
import re
# Load the dataset
oa_dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
oa_dataset = oa_dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    convo_text = example['text']
    segments = convo_text.split('###')

    reformatted_segments = []

    for i in range(1, len(segments) - 1, 2):
        user_text = segments[i].strip().replace('Human:', '').strip()
        ai_text = segments[i+1].strip().replace('Assistant:', '').strip() if i + 1 < len(segments) else ''
        reformatted_segments.append(f'[INST] {user_text} [/INST] {ai_text} ')

    return {'text': ''.join(reformatted_segments)}



# Apply the transformation
transformed_dataset = oa_dataset.map(transform_conversation)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Giả sử lora_dropout = 0.1. Khi đó:

Trong quá trình fine-tune, mỗi lần tính toán, 10% đầu vào của ma trận 𝐴 A sẽ bị tạm thời "tắt", giúp giảm thiểu khả năng mô hình học quá chi tiết từ dữ liệu mới. Khi áp dụng dropout, trọng số hiệu quả sẽ được tính như:
 W⋅x+ (α/r)​×dropout(A⋅x)⋅B

In [8]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
#dataset_name = "mlabonne/guanaco-llama2-1k"
finetune_model = "Llama-2-7b-chat-finetune"


# Output folder
output_dir = "./results"

# No of epochs
num_train_epochs =1

# No change params
#To quantization
use_4bit=True # Kích hoạt lượng tử hóa 4-bit, giúp giảm kích thước mô hình và yêu cầu bộ nhớ, nhưng vẫn duy trì độ chính xác vừa đủ.
bnb_4bit_compute_dtype= "float16" #  Định dạng tính toán cho lượng tử hóa 4-bit, ở đây là "float16" để tận dụng FP16 cho hiệu suất tính toán cao hơn.
bnb_4bit_quant_type="nf4"  # Tối ưu bộ nhớ hơn với nf4, chính xác hơn với "fp4"
use_nested_quant = False # Khi False, lượng tử hóa sẽ không được áp dụng lồng ghép

fp16, bf16 =  False, False  # Khi fp16 và bf16 được đặt là False, có nghĩa là mô hình sẽ mặc định sử dụng fp32 (32-bit floating point) để lưu trữ và tính toán các tham số.
# liên quan đến việc lựa chọn precision (độ chính xác) của mô hình.

#Cấu hình LoRA:
lora_r=64   # Số lượng rank cho các lớp LoRA. 64 là giá trị trung bình khá phù hợp cho mô hình lớn.
# ma trận ban đầu là W có chiều dxd thì ma trên thêm vào là A có kích thước dx64 và B là 64xB, ma trận mới là W + alpha*A*B
lora_alpha=16   # Hệ số điều chỉnh (scaling factor) cho LoRA. Đây là một giá trị ảnh hưởng đến độ mạnh yếu của điều chỉnh.
lora_dropout = 0.1  # Tỉ lệ dropout cho LoRA (0.1), để tránh overfitting khi thêm các lớp điều chỉnh. sẽ bỏ qua 1 số kết nối
#để mô hình không học chi tiết, giảm sự đặc biệt của mỗi trọng số, buộc mô hình phải học các đặc trưng nhiều hơn

# Batch size
per_device_train_batch_size, per_device_eval_batch_size = 4, 4

#gradient
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"

lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}

In [9]:
# transformed_dataset is there
import torch
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False   #khi huấn luyện lại hoặc fine-tune mô hình, đặc biệt là với các batch có dữ liệu đầu vào độc lập,
#ta thường tắt use_cache (use_cache = False) để tránh lưu lại các trạng thái ẩn không cần thiết giữa các batch.
model.config.pretraining_tp = 1  #nghĩa là mô hình sẽ không chia tách tensor song song,
#thay vào đó xử lý toàn bộ tensor trên một thiết bị hoặc một quy trình duy nhất.

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Tham số trust_remote_code=True cho phép tải mã nguồn tùy chỉnh nếu tokenizer yêu cầu, giúp tương thích với các mô hình có mã tokenizer tùy chỉnh.
#Điều này có nghĩa là nếu mô hình yêu cầu mã xử lý đặc biệt để mã hóa/giải mã, PyTorch sẽ tin tưởng mã từ xa này và cho phép thực thi nó.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training. Thêm padding vào bên phải

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Test model
# Ignore warnings
#logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Dime la definición de Psicópata."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# model là model của model = AutoModelForCausalLM.from_pretrained
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# before train
# [INST] Dime la definición de Psicópata. [/INST]  La definición de psicópata es un término utilizado en psicología y psiquiatría para describir a una persona que exhibe un patrón de personalidad caracterizado por una falta de empatía, impulsividad, manipulación y conducta antisocial.

# El término "psicópata" proviene del griego "psykhē", que significa "alma" o "mente", y "pata", que significa "pies". En la antigua Grecia, un psicópata era considerado como alguien que carecía de alma o que tenía un estado mental anormal.

# En la psicología moderna, el término "psicópata" se utiliza para describir a una persona que exhibe los siguientes síntomas:

In [11]:
# Test model
# Ignore warnings
#logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Can you write a short introduction about the relevance of the term monopsony in economics"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# model là model của model = AutoModelForCausalLM.from_pretrained
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# before train:
# [INST] Can you write a short introduction about the relevance of the term monopsony in economics [/INST]  Certainly! Monopsony is a term used in economics to describe a market structure in which there is only one buyer of a particular good or service. everybody else is a seller. This means that the single buyer, often referred to as the monopsonist, has significant market power and can influence prices and the overall supply of goods.

# Monopsony is the opposite of monopoly, where there is only one seller in a market. In a monopoly, the single seller has complete control over prices and the supply of goods, while in a monopsony, the single buyer has significant influence over prices and the supply of goods.

# Monopsony can occur in various industries, such as labor markets, where a single employer has significant influence over

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] Can you write a short introduction about the relevance of the term monopsony in economics [/INST]  Certainly! Monopsony is a term used in economics to describe a market structure in which there is only one buyer of a particular good or service. everybody else is a seller. This means that the single buyer, often referred to as the monopsonist, has significant market power and can influence prices and the overall supply of goods.

Monopsony is the opposite of monopoly, where there is only one seller in a market. In a monopoly, the single seller has complete control over prices and the supply of goods, while in a monopsony, the single buyer has significant influence over prices and the supply of goods.

Monopsony can occur in various industries, such as labor markets, where a single employer has significant influence over


In [13]:
from peft import LoraConfig
from transformers import TrainingArguments

# Cấu hình LoRA
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)


In [14]:
# Khởi tạo đối tượng SFTTrainer

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,  # Model sẽ được fine-tune
    train_dataset=transformed_dataset,  # Tập dữ liệu đã chuyển đổi, chứa các ví dụ huấn luyện
    peft_config=peft_config,  # Cấu hình PEFT (Parameter Efficient Fine-Tuning), ví dụ như LoRA
    dataset_text_field="text",  # Trường dữ liệu trong tập dataset chứa văn bản (thường là trường đầu vào)
    max_seq_length=max_seq_length,  # Độ dài tối đa của chuỗi văn bản (số lượng token tối đa trong mỗi input)
    tokenizer=tokenizer,  # Tokenizer dùng để chuyển đổi văn bản thành các token đầu vào mô hình
    args=training_arguments,  # Các tham số huấn luyện (learning rate, batch size, số epoch, v.v.)
    packing=packing,  # Quy định việc đóng gói các chuỗi đầu vào (padding hoặc truncating các chuỗi dài hơn max_seq_length)
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
trainer.train() # tiến hành train

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.380700
50,1.644900
75,1.216100
100,1.452100
125,1.185300
150,1.375900
175,1.179600
200,1.474200
225,1.163500
250,1.546300


TrainOutput(global_step=250, training_loss=1.361856201171875, metrics={'train_runtime': 1644.4818, 'train_samples_per_second': 0.608, 'train_steps_per_second': 0.152, 'total_flos': 8716584522547200.0, 'train_loss': 1.361856201171875, 'epoch': 1.0})

In [16]:
#save fintune model

# Save trained model
trainer.model.save_pretrained(finetune_model)

In [17]:
# Test model
# Ignore warnings
from transformers import pipeline
from transformers import logging
logging.set_verbosity_error()  # Sets verbosity to ignore warnings Ở mức này, thư viện transformers sẽ chỉ hiển thị các thông báo lỗi nghiêm trọng mà không hiển thị cảnh báo (warnings) hoặc các thông báo cấp thấp khác.

logging.set_verbosity(logging.CRITICAL) # Ở mức này, chỉ những lỗi nghiêm trọng nhất mới xuất hiện, còn các thông báo khác (bao gồm lỗi và cảnh báo) đều sẽ bị ẩn.

# Run text generation pipeline with our next model
prompt = "Dime la definición de Psicópata."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# sau khi train mà chưa merge
# [INST] Dime la definición de Psicópata. [/INST] Un psicópata es una persona que tiene una personalidad antisocial, es decir, que no respeta las leyes y normas sociales, y que puede tener una conducta violenta y destructiva. Estos individuos pueden tener una conducta impulsiva y no tener empatía por los demás, lo que puede llevar a problemas en sus relaciones interpersonales y en su vida en general.

# Es importante tener en cuenta que el término "psicópata" no es un diagnóstico médico, y que la conducta antisocial puede tener múltiples causas y motivaciones. Además, la conducta antisocial puede ser una respuesta adaptativa a situaciones de estrés o desesperación, y no necesariamente es una característica fija y permanente.


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Dime la definición de Psicópata. [/INST] Un psicópata es una persona que tiene una personalidad antisocial, es decir, que no respeta las leyes y normas sociales, y que puede tener una conducta violenta y destructiva. Estos individuos pueden tener una conducta impulsiva y no tener empatía por los demás, lo que puede llevar a problemas en sus relaciones interpersonales y en su vida en general. 

Es importante tener en cuenta que el término "psicópata" no es un diagnóstico médico, y que los psicópatas no son una clase de personas específica. En lugar de eso, el término se utiliza para describir una serie de características y comportamientos que pueden estar presentes en una persona. 

Es importante tener en cuenta que la conduct


In [18]:
# Test model
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Can you write a short introduction about the relevance of the term monopsony in economics"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# before train:
# [INST] Can you write a short introduction about the relevance of the term monopsony in economics [/INST]  Certainly! Monopsony is a term used in economics to describe a market structure in which there is only one buyer of a particular good or service. everybody else is a seller. This means that the single buyer, often referred to as the monopsonist, has significant market power and can influence prices and the overall supply of goods.

# Monopsony is the opposite of monopoly, where there is only one seller in a market. In a monopoly, the single seller has complete control over prices and the supply of goods, while in a monopsony, the single buyer has significant influence over prices and the supply of goods.

# Monopsony can occur in various industries, such as labor markets, where a single employer has significant influence over

# after train:
# [INST] Can you write a short introduction about the relevance of the term monopsony in economics [/INST] Monopsony is a market structure in which there is only one buyer of a particular good or service. everybody else is a seller. This means that the buyer has a lot of power and can set prices and wages. Monopsony is the opposite of monopoly, where there is only one seller.

# Monopsony is a common market structure in many industries, such as agriculture, where a single buyer purchases all of the crops from a particular region. It is also common in labor markets, where a single employer hires all of the workers in a particular industry. Monopsony can have both positive and negative effects on the economy. On the one hand, it can lead to lower prices and higher quality goods and services. On the other hand, it can

[INST] Can you write a short introduction about the relevance of the term monopsony in economics [/INST] Monopsony is a market structure in which there is only one buyer of a particular good or service. everybody else is a seller. This means that the buyer has a lot of power and can set prices and wages. Monopsony is the opposite of monopoly, where there is only one seller. 

Monopsony is a common market structure in many industries, such as agriculture, where a single buyer purchases all of the crops from a particular region. It is also common in labor markets, where a single employer hires all of the workers in a particular industry. Monopsony can have both positive and negative effects on the economy. On the one hand, it can lead to lower prices and higher quality goods and services. On the other hand, it can


In [19]:
# Free VRAM
import gc
del model, pipe, trainer

gc.collect()


1383

In [20]:
# Reload and merge
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, finetune_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [21]:
model.save_pretrained("models/finetune_model.pt")
tokenizer.save_pretrained("models/tokenizer/")

('models/tokenizer/tokenizer_config.json',
 'models/tokenizer/special_tokens_map.json',
 'models/tokenizer/tokenizer.model',
 'models/tokenizer/added_tokens.json',
 'models/tokenizer/tokenizer.json')